In [1]:
from typing import Tuple, List, Dict
from dataclasses import dataclass

triples: List[Tuple[str, str, str]] = []

create_match = lambda tr: lambda s, p, o: [t for t in tr if (not s or t[0] == s) and (not p or t[1] == p) and (not o or t[2] == o)]
match = lambda s = None, p = None, o = None: create_match(triples)(s, p, o)
match_first = lambda s = None, p = None, o = None : next(iter(match(s, p, o) or []), (None, None, None))


In [2]:
import shlex 
    
filenames = ['emmo.nt']
for filename in filenames:
    with open(filename, 'r') as file:            
        lines = shlex.split(file.read());        
        for i in range(0, len(lines), 4):
            triples.append((lines[i], lines[i+1], lines[i+2])) 
          

In [3]:
(s, p, o) = match_first('<http://emmo.info/emmo#EMMO_705f27ae_954c_4f13_98aa_18473fc52b25>')
print (p, o)

<http://www.w3.org/2004/02/skos/core#prefLabel> etymology@en


In [4]:
from ipytree import Tree, Node
from ipywidgets import HBox, HTML, Output, Layout
import html
from namespaces import RDFS, SKOS, EMMO

a = Output(layout=Layout(min_width='50%'))
nodemap = {}

style = """
<style>
body{
  font-family: 'Nunito', sans-serif;
  padding: 50px;
}
p.note {
    background: #fffffa;
    border: 2px solid #aaaa55;
}
.card{
    border-radius: 4px;
    background: #fff;
    box-shadow: 0 6px 10px rgba(0,0,0,.08), 0 0 6px rgba(0,0,0,.05);
      transition: .3s transform cubic-bezier(.155,1.105,.295,1.12),.3s box-shadow,.3s -webkit-transform cubic-bezier(.155,1.105,.295,1.12);
  padding: 14px 80px 18px 36px;
  cursor: pointer;
}

.card:hover{
     transform: scale(1.05);
  box-shadow: 0 10px 20px rgba(0,0,0,.12), 0 4px 8px rgba(0,0,0,.06);
}

.card h3{
  font-weight: 600;
}

.card img{
  position: absolute;
  top: 20px;
  right: 15px;
  max-height: 120px;
}

.card-1{
  background-image: url(https://avatars.githubusercontent.com/u/51194521?s=200&v=4);
      background-repeat: no-repeat;    
    background-position: right 10px top 10px;
    background-size: 80px;
}

@media(max-width: 990px){
  .card{
    margin: 20px;
  }
} 
</style>
"""

def handle_click(event):
    concept = nodemap[event['owner']._id]
    a.clear_output()
    with a:
        (_,_,desc) = match_first(concept, EMMO.elucidation, None)
        comments = list(match(concept, RDFS.comment, None))
        comments = "\n".join(list([html.unescape(c[2]).replace('@en','\n') for c in comments])).replace('\\n','<br/>')
        desc = html.unescape(desc).replace('@en','') if desc else ""
        (_, _, etymology) = match_first(concept, EMMO.etymology, None)
        etymology = html.unescape(etymology).replace('@en','').replace('\\n','\n') if etymology else ""
        display(HTML(value=f"""{style}
        <link href="https://fonts.googleapis.com/css?family=Nunito:200,300,400,600,700,900" rel="stylesheet">

        <div class="container">
          <div class="row">
            <div class="col-md-4">
              <div class="card card-1">
                <h3>{event['owner'].name}</h3>
                <p><b>{html.escape(concept)}</b><br/></p>                
                <p class="note">
                <h4>Elucidation</h4>
                {desc}</p>                
                <p class="note">
                <h4>Comments</h4>
                {comments}</p>
                <p class="note">
                <h4>Etymology</h4>
                {etymology}</p>
              </div>
            </div>
          </div>
        </div>
        """))
        
def gen_tree(root):
    l = []
    
    # Recursively generate and append sub-nodes
    for (s,_,_) in match(None, RDFS.subClassOf, root):        
        l.append(gen_tree(s))
        
    # Find the prefLabel (node name)
    (_,_,lbl) = match_first(root, SKOS.prefLabel, None)
    lbl = lbl.replace('@en','')
    if l:
        node = Node(lbl, l, opened=False, icon_style="success")
    else:
        node = Node(lbl, l, opened=False, icon="leaf", icon_style="success")
        
    # Add observer to handle interactions
    node.observe(handle_click, 'selected')
    nodemap[node._id] = root
    return node

emmo_node = gen_tree('<http://emmo.info/emmo#EMMO_802d3e92_8770_4f98_a289_ccaaab7fdddf>')

In [5]:
tree=Tree(layout=Layout(min_width='50%'))

tree.add_node(emmo_node)
box = HBox(children=[tree, a], layout=Layout(align_items='stretch', width='100%'))

display (box)


Output()